In [1]:
from twarc import Twarc
import os
os.chdir(r'C:\Users\Leonardo\OneDrive\Documents\TU_Delft\CodingProjects\PoliceBrutality')

In [9]:
import json
t = Twarc(app_auth=True)

keywords = 'racism OR george floyd OR police brutality OR black lives matter OR killing black OR blacklivesmatter OR police violence OR ahmaud arbery OR #blacklivesmatter OR #blm OR #JusticeForGeorgeFloyd OR #ICantBreathe OR #GeorgeFloyd -filter:retweets'

with open('data/raw/tweet_data3.jsonl', 'w', encoding='utf-8') as f:
    for tweet in t.search(keywords):
        json.dump(tweet, f)
        f.write('\n')

In [ ]:
'racism OR george floyd OR police brutality OR black lives matter OR killing black OR blacklivesmatter OR police violence OR ahmaud arbery OR #blacklivesmatter OR #blm OR #JusticeForGeorgeFloyd OR #ICantBreathe OR #GeorgeFloyd -filter:retweets' > tweets_data4.jsonl

In [5]:
import json
t = Twarc(app_auth=True)

keywords = 'racism OR george floyd OR police brutality OR black lives matter OR killing black OR blacklivesmatter OR police violence OR #blacklivesmatter OR #blm OR #JusticeForGeorgeFloyd OR #ICantBreathe OR #GeorgeFloyd'

with open('data/raw/tweet_data_rt.jsonl', 'w', encoding='utf-8') as f:
    for tweet in t.filter(keywords):
        json.dump(tweet, f)
        f.write('\n')

In [7]:
import pandas as pd
df = pd.read_json('data/raw/tweet_data2.jsonl', lines=True)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#Twitter credentials for the app
consumer_key = 'uFZGUI4934oUJ6OzOrneWtiva'
consumer_secret = 'Zm7B75DdwBDx5rLCMhLChQXhWEEfhqKf00gJFXYL7crFoDJi9A'
access_key= '1200171740262998017-2sofVaYjzxkX4ZHjCLjXMezWrK5vZd'
access_secret = 'izbr2xHkbncDqxg17Fmimt45KqKrdCRKYSPsFASSEEknF'

#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#file location changed to "data/telemedicine_data_extraction/" for clearer path
police_brutality_tweets = "data/raw/tweet_data.csv"

#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
        'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']

#set two date variables for date range
# start_date = '2020-05-22'
start_date = '2018-05-22'

# end_date = '2018-10-31'

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)


#mrhod clean_tweets()
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)

    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)


    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

#method write_tweets()
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date, tweet_mode='extended').pages(10):
        for status in page:
            new_entry = []
            status = status._json

            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'en':
                continue

            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue


            #tweepy preprocessing called for basic preprocessing
            clean_text = p.clean(status['full_text'])

            #call clean_tweet method for extra preprocessing
            filtered_tweet=clean_tweets(clean_text)

            #pass textBlob method for sentiment calculations
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment

            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity

            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['full_text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]

            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])

            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)

            # hashtags and mentions are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)

            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)

            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)

            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

#declare keywords as a query for three categories
police_brutality_keywords = 'racism OR george floyd OR police brutality OR black lives matter OR killing black OR blacklivesmatter OR police violence OR ahmaud arbery OR #blacklivesmatter OR #blm OR #JusticeForGeorgeFloyd OR #ICantBreathe OR #GeorgeFloyd -filter:retweets'

#call main method passing keywords and file path
write_tweets(police_brutality_keywords,  police_brutality_tweets)

In [2]:
import tweepy 
from tweepy import Stream
from tweepy.streaming import StreamListener 
import json
import time
import datetime


class MyListener(StreamListener):
    def __init__(self, api, file_name_base):
        self.tweet_list=[]
        self.count = 1
        self.file_name_base = file_name_base
       
    def on_data(self, data):
        try:
            j = json.loads(data)
            self.tweet_list.append(j)
            n = len(self.tweet_list)
  
            if (n % 100 == 0):
                ts = time.time()
                stamp = datetime.datetime.fromtimestamp(ts).strftime('%m%d%H%M%S')
                file_name = self.file_name_base + stamp + '.json'
                with open(file_name , 'w') as f:
                    json.dump(self.tweet_list, f)
#                     f.write('\n')
                print("Output File", self.count)
                self.count = self.count + 1
                self.tweet_list = []
            return True
        except BaseException as e:
            print("Error on_data:", str(e))
        return True
 
    def on_error(self, status):
        print("Error",status)
        return False

class Credentials:
    def __init__(self, access_token, access_token_secret, consumer_key, 
        consumer_secret):
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret


def mine_tweets(credentials, search_terms, file_name):
    auth = tweepy.OAuthHandler(credentials.consumer_key, credentials.consumer_secret)
    auth.set_access_token(credentials.access_token, credentials.access_token_secret)

    api = tweepy.API(auth)

    twitter_stream = Stream(auth, MyListener(auth, file_name))

    print("Starting up!")

    while(True):
        try:
            twitter_stream.filter(track=search_terms)
        except:
            time.sleep(30)
            print("program restart")

In [ ]:
import tweepy 
from tweepy import Stream
from tweepy.streaming import StreamListener 
import json
import time
import datetime
# from twitter_miner import *

ACCESS_TOKEN = '1200171740262998017-2sofVaYjzxkX4ZHjCLjXMezWrK5vZd'
ACCESS_TOKEN_SECRET = 'izbr2xHkbncDqxg17Fmimt45KqKrdCRKYSPsFASSEEknF'
CONSUMER_KEY = 'uFZGUI4934oUJ6OzOrneWtiva'
CONSUMER_SECRET = 'Zm7B75DdwBDx5rLCMhLChQXhWEEfhqKf00gJFXYL7crFoDJi9A'

text_list = ['racism', 'george floyd', 'police brutality', 'black lives matter', 
             'killing black', 'blacklivesmatter', 'police violence', 'ahmaud arbery'] 
hashtag_list = ['#blacklivesmatter', '#blm', '#JusticeForGeorgeFloyd',' #ICantBreathe', '#GeorgeFloyd']
mentions_list = []
from_list = []
to_list = []

full_list = text_list
full_list.extend(hashtag_list)
full_list.extend(mentions_list)
full_list.extend(from_list)
full_list.extend(to_list)

credentials = Credentials(ACCESS_TOKEN, ACCESS_TOKEN_SECRET,
    CONSUMER_KEY, CONSUMER_SECRET)

file_name_base = 'data/raw/AMIR/' #enter path where you want the file saved to

mine_tweets(credentials, full_list, file_name_base)

Starting up!
Output File 1
Output File 2
Output File 3
Output File 4
Output File 5


In [ ]:
twarc filter blacklivesmatter,blm > rt_tweets.jsonl

In [ ]:
twarc filter blacklivesmatter,blm > rt_tweets.jsonl

In [ ]:
twarc search 'blacklivesmatter OR police OR violence OR #blacklivesmatter OR #blm' > past_tweets.jsonl

In [ ]:
twarc search '#blacklivesmatter OR #blm' > past_tweets.jsonl

In [13]:
import pandas as pd
pd.read_csv('rt_tweets.txt', sep=" ", header=None)

In [16]:
import json

with open('rt_tweets.jsonl') as tweet_data:
    json_data = json.load(tweet_data)

In [30]:
df = pd.read_json('tweets_test2.jsonl', lines=True)